Deep Learning   

In [2]:
!pip install torch 

!pip install pandas numpy matplotlib seaborn plotly scikit-learn

  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 2.6/216.1 MB 18.9 MB/s eta 0:00:12
   - -------------------------------------- 7.3/216.1 MB 23.8 MB/s eta 0:00:09
   - -------------------------------------- 7.9/216.1 MB 15.2 MB/s eta 0:00:14
   - -------------------------------------- 8.9/216.1 MB 12.0 MB/s eta 0:00:18
   - -------------------------------------- 10.0/216.1 MB 10.5 MB/s eta 0:00:20
   -- ------------------------------------- 11.3/216.1 MB 9.6 MB/s eta 0:00:22
   -- ------------------------------------- 15.5/216.1 MB 1

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
train_df = pd.read_csv("train.csv")
X = train_df.iloc[:, :55].values  # First 55 columns
y = train_df.iloc[:, 55:].values  # Last 10 columns (targets)

# Normalize inputs (important for deep learning)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split for validation
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=64)

# Model
class FuelNet(nn.Module):
    def __init__(self):
        super(FuelNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(55, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),
            nn.Linear(64, 10)  # 10 targets
        )

    def forward(self, x):
        return self.model(x)

model = FuelNet()


In [12]:
import torch.optim as optim

criterion = nn.L1Loss()  # MAE (similar to MAPE in behavior)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(1000):
    model.train()
    train_loss = 0.0
    for Xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(Xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for Xb, yb in val_loader:
            preds = model(Xb)
            loss = criterion(preds, yb)
            val_loss += loss.item()

    print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")


Epoch 1, Train Loss: 0.2725, Val Loss: 0.2450
Epoch 2, Train Loss: 0.2740, Val Loss: 0.2428
Epoch 3, Train Loss: 0.2795, Val Loss: 0.2442
Epoch 4, Train Loss: 0.2744, Val Loss: 0.2436
Epoch 5, Train Loss: 0.2672, Val Loss: 0.2432
Epoch 6, Train Loss: 0.2664, Val Loss: 0.2435
Epoch 7, Train Loss: 0.2631, Val Loss: 0.2426
Epoch 8, Train Loss: 0.2688, Val Loss: 0.2431
Epoch 9, Train Loss: 0.2690, Val Loss: 0.2436
Epoch 10, Train Loss: 0.2649, Val Loss: 0.2428
Epoch 11, Train Loss: 0.2604, Val Loss: 0.2420
Epoch 12, Train Loss: 0.2674, Val Loss: 0.2428
Epoch 13, Train Loss: 0.2639, Val Loss: 0.2414
Epoch 14, Train Loss: 0.2691, Val Loss: 0.2397
Epoch 15, Train Loss: 0.2652, Val Loss: 0.2403
Epoch 16, Train Loss: 0.2574, Val Loss: 0.2405
Epoch 17, Train Loss: 0.2604, Val Loss: 0.2403
Epoch 18, Train Loss: 0.2651, Val Loss: 0.2416
Epoch 19, Train Loss: 0.2694, Val Loss: 0.2401
Epoch 20, Train Loss: 0.2707, Val Loss: 0.2380
Epoch 21, Train Loss: 0.2595, Val Loss: 0.2383
Epoch 22, Train Loss: 

KeyboardInterrupt: 

In [6]:
# Load test.csv
test_df = pd.read_csv("test.csv")

# Separate ID and features
test_ids = test_df.iloc[:, 0]  # First column is ID
X_test = test_df.iloc[:, 1:].values  # Remaining 55 features

# Scale using the same scaler as training
X_test_scaled = scaler.transform(X_test)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

# Predict with trained model
model.eval()
with torch.no_grad():
    test_preds = model(X_test_tensor).numpy()

# Create submission DataFrame
submission = pd.DataFrame(test_preds, columns=[f'BlendProperty{i+1}' for i in range(10)])
submission.insert(0, "ID", test_ids)  # Insert ID as the first column

# Save to CSV
submission.to_csv("submission.csv", index=False)


attempt 2

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

df = pd.read_csv("train.csv")
X = df.iloc[:, :55].values       # Input features (columns 0–54)
y = df.iloc[:, 55:].values       # Target values (columns 55–64)
# Feature scaling
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# Target scaling
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)


In [15]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class FuelDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class FuelNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(55, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)


In [20]:
from sklearn.metrics import mean_absolute_percentage_error
import torch.optim as optim

kf = KFold(n_splits=5, shuffle=True, random_state=42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_df = pd.read_csv("test.csv")
X_test_scaled = scaler_X.transform(test_df.iloc[:, 1:].values)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
final_preds = np.zeros((500, 10))

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    X_train, y_train = X_scaled[train_idx], y_scaled[train_idx]
    X_val, y_val = X_scaled[val_idx], y_scaled[val_idx]

    train_dataset = FuelDataset(X_train, y_train)
    val_dataset = FuelDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64)

    model = FuelNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
    criterion = nn.MSELoss()

    best_val_mape = float("inf")
    patience, wait = 10, 0

    for epoch in range(1000):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
        scheduler.step()

        # Validation
        model.eval()
        val_preds, val_targets = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                val_preds.append(preds.cpu().numpy())
                val_targets.append(yb.cpu().numpy())
        
        val_preds = np.vstack(val_preds)
        val_targets = np.vstack(val_targets)

        # Inverse scale targets
        val_preds_inv = scaler_y.inverse_transform(val_preds)
        val_targets_inv = scaler_y.inverse_transform(val_targets)

        mape = mean_absolute_percentage_error(val_targets_inv, val_preds_inv)
        print(f"[Fold {fold+1}] Epoch {epoch+1} - Val MAPE: {mape:.4f}")

        # Early stopping
        if mape < best_val_mape:
            best_val_mape = mape
            best_model = model.state_dict()
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                break

    # Load best model for inference
    model.load_state_dict(best_model)
    model.eval()
    with torch.no_grad():
        fold_preds = model(X_test_tensor).cpu().numpy()
        final_preds += scaler_y.inverse_transform(fold_preds) / kf.n_splits


[Fold 1] Epoch 1 - Val MAPE: 2.4448
[Fold 1] Epoch 2 - Val MAPE: 3.7573
[Fold 1] Epoch 3 - Val MAPE: 4.0852
[Fold 1] Epoch 4 - Val MAPE: 4.3686
[Fold 1] Epoch 5 - Val MAPE: 4.1321
[Fold 1] Epoch 6 - Val MAPE: 4.2867
[Fold 1] Epoch 7 - Val MAPE: 4.0076
[Fold 1] Epoch 8 - Val MAPE: 4.1436
[Fold 1] Epoch 9 - Val MAPE: 3.7571
[Fold 1] Epoch 10 - Val MAPE: 3.5929
[Fold 1] Epoch 11 - Val MAPE: 3.4571
[Fold 2] Epoch 1 - Val MAPE: 2.3974
[Fold 2] Epoch 2 - Val MAPE: 2.7774
[Fold 2] Epoch 3 - Val MAPE: 2.7279
[Fold 2] Epoch 4 - Val MAPE: 2.5919
[Fold 2] Epoch 5 - Val MAPE: 2.4364
[Fold 2] Epoch 6 - Val MAPE: 2.4276
[Fold 2] Epoch 7 - Val MAPE: 2.4276
[Fold 2] Epoch 8 - Val MAPE: 2.4771
[Fold 2] Epoch 9 - Val MAPE: 2.5331
[Fold 2] Epoch 10 - Val MAPE: 2.5422
[Fold 2] Epoch 11 - Val MAPE: 2.5717
[Fold 3] Epoch 1 - Val MAPE: 2.7932
[Fold 3] Epoch 2 - Val MAPE: 3.4225
[Fold 3] Epoch 3 - Val MAPE: 3.2599
[Fold 3] Epoch 4 - Val MAPE: 3.1470
[Fold 3] Epoch 5 - Val MAPE: 2.9399
[Fold 3] Epoch 6 - Val M

In [11]:
submission = pd.DataFrame(final_preds, columns=[f'BlendProperty{i+1}' for i in range(10)])
submission.insert(0, "ID", test_df["ID"])
submission.to_csv("submission.csv", index=False, float_format="%.6f")


In [18]:
pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm

# Load data
train_df = pd.read_csv("train.csv")
X = train_df.iloc[:, :55].values
y = train_df.iloc[:, 55:].values

# Scale inputs and targets
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Load test set
test_df = pd.read_csv("test.csv")
test_ids = test_df.iloc[:, 0]
X_test = test_df.iloc[:, 1:].values
X_test_scaled = scaler_X.transform(X_test)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to("cuda" if torch.cuda.is_available() else "cpu")

# Dataset class
class FuelDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Model class
class FuelNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(55, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),
            nn.Linear(128, 10)
        )
    def forward(self, x):
        return self.net(x)

# Training config
kf = KFold(n_splits=5, shuffle=True, random_state=42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
final_preds = np.zeros((500, 10))

for fold, (train_idx, val_idx) in enumerate(kf.split(X_scaled)):
    X_train, y_train = X_scaled[train_idx], y_scaled[train_idx]
    X_val, y_val = X_scaled[val_idx], y_scaled[val_idx]

    train_dataset = FuelDataset(X_train, y_train)
    val_dataset = FuelDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64)

    model = FuelNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
    criterion = nn.MSELoss()

    best_val_mape = float("inf")
    patience = 500
    wait = 0
    min_delta = 1e-4

    for epoch in tqdm(range(1000), desc=f"Fold {fold+1}"):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
        scheduler.step()

        # Validation
        model.eval()
        val_preds, val_targets = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                preds = model(xb)
                val_preds.append(preds.cpu().numpy())
                val_targets.append(yb.cpu().numpy())

        val_preds = np.vstack(val_preds)
        val_targets = np.vstack(val_targets)
        val_preds_inv = scaler_y.inverse_transform(val_preds)
        val_targets_inv = scaler_y.inverse_transform(val_targets)

        mape = mean_absolute_percentage_error(val_targets_inv, val_preds_inv)
        if (epoch + 1) % 10 == 0 or (best_val_mape - mape) > min_delta:
            print(f"[Fold {fold+1}] Epoch {epoch+1} - Val MAPE: {mape:.4f}")


        if (best_val_mape - mape) > min_delta:
            best_val_mape = mape
            best_model = model.state_dict()
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print(f"[Fold {fold+1}] Early stopping at epoch {epoch+1}")
                break

    # Predict on test set using best model
    model.load_state_dict(best_model)
    model.eval()
    with torch.no_grad():
        fold_preds = model(X_test_tensor).cpu().numpy()
        final_preds += scaler_y.inverse_transform(fold_preds) / kf.n_splits

# Create submission
submission = pd.DataFrame(final_preds, columns=[f'BlendProperty{i+1}' for i in range(10)])
submission.insert(0, "ID", test_ids)
submission.to_csv("submission.csv", index=False, float_format="%.6f")


Fold 1:   0%|          | 2/1000 [00:00<01:18, 12.73it/s]

[Fold 1] Epoch 1 - Val MAPE: 3.0039


Fold 1:   1%|          | 12/1000 [00:00<01:07, 14.74it/s]

[Fold 1] Epoch 10 - Val MAPE: 5.6670


Fold 1:   2%|▏         | 22/1000 [00:01<01:05, 14.99it/s]

...

[Fold 5] Epoch 860 - Val MAPE: 1.6560


Fold 5:  87%|████████▋ | 872/1000 [00:56<00:08, 15.56it/s]

[Fold 5] Epoch 870 - Val MAPE: 1.6258


Fold 5:  88%|████████▊ | 876/1000 [00:56<00:07, 15.53it/s]

[Fold 5] Early stopping at epoch 877
